In [1]:
# Import libraries
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import altair_saver 

In [2]:

os.chdir('C://Users/pmarshal/Documents/Climate-Outlook/monthly-climate')
os.getcwd()

'C:\\Users\\pmarshal\\Documents\\Climate-Outlook\\monthly-climate'

# Snowpack 

Below you will graphs, maps, and images describing snowpack conditions in the water supply areas during the 2023-24 winter season. 

In [3]:
orchid = pd.read_csv('data/orchid_snow.csv', parse_dates=['date']) 
orchid = orchid.dropna(subset=['depth'])
orchid['year'] = pd.DatetimeIndex(orchid['date']).year
orchid['month'] = pd.DatetimeIndex(orchid['date']).month
orchid['day'] = pd.DatetimeIndex(orchid['date']).day
orchid['DOY'] = pd.DatetimeIndex(orchid['date']).dayofyear
# Assuming 'wtr_day_year' is currently in integer format
orchid['wtr_day_year'] = pd.to_datetime('2023-01-01') + pd.to_timedelta(orchid['wtr_day_year'] - 93, unit='D')


In [4]:
# Claculate statistics on mean temp column
orchid['mean'] = orchid.groupby('wtr_day_year')['depth'].transform('mean').round(1)
orchid['max'] = orchid.groupby('wtr_day_year')['depth'].transform('max')
orchid['min'] = orchid.groupby('wtr_day_year')['depth'].transform('min')
orchid['std'] = orchid.groupby('wtr_day_year')['depth'].transform('std')
orchid['sem'] = orchid.groupby('wtr_day_year')['depth'].transform('sem')
orchid['ci95_hi'] = orchid['mean'] + 1.96* orchid['sem']
orchid['ci95_lo'] = orchid['mean'] - 1.96* orchid['sem']
orchid['percentile_25th'] = orchid.groupby('wtr_day_year')['depth'].transform(lambda x: np.percentile(x, 25))
orchid['percentile_75th'] = orchid.groupby('wtr_day_year')['depth'].transform(lambda x: np.percentile(x, 75))
orchid.tail()

,date,depth,wtr_year,wtr_day_year,year,month,day,DOY,mean,max,min,std,sem,ci95_hi,ci95_lo,percentile_25th,percentile_75th
6263,2024-12-27,186.1,2025,2022-12-27,2024,12,27,362,191.0,359.8,23.9,83.290215,20.200844,230.593655,151.406345,154.3,241.1
6264,2024-12-28,203.0,2025,2022-12-28,2024,12,28,363,193.4,353.7,23.4,83.393956,20.226005,233.042970,153.757030,153.4,244.7
6265,2024-12-29,203.1,2025,2022-12-29,2024,12,29,364,195.4,352.4,24.3,82.792861,20.080218,234.757228,156.042772,160.2,240.2
6266,2024-12-30,210.3,2025,2022-12-30,2024,12,30,365,198.9,353.0,24.7,84.387017,20.466858,239.015041,158.784959,183.1,249.6
6267,2024-12-31,208.5,2025,2022-12-31,2024,12,31,366,201.3,350.3,23.6,83.378048,20.222147,240.935408,161.664592,198.5,243.7


In [5]:
#Percent normal
Percent = (208.5/201.3)*100
Percent

103.57675111773472

In [6]:
# filter to remove non snow season
orchid = orchid[(orchid['wtr_day_year'] >= '2022-10-01') & (orchid['wtr_day_year'] <= '2023-08-15')]

In [7]:
last = orchid.tail(1)
last

,date,depth,wtr_year,wtr_day_year,year,month,day,DOY,mean,max,min,std,sem,ci95_hi,ci95_lo,percentile_25th,percentile_75th
6267,2024-12-31,208.5,2025,2022-12-31,2024,12,31,366,201.3,350.3,23.6,83.378048,20.222147,240.935408,161.664592,198.5,243.7


In [8]:
# Create dataframe with data for the current year
orchid_2025 = orchid.loc[orchid['wtr_year'] == 2025]
orchid_2010 = orchid.loc[orchid['wtr_year'] == 2010]

# Create dataframe with historical data (i.e. not == current year)
orchid_past = orchid.loc[orchid['wtr_year'] != 2025]
orchid_stats = orchid_past.loc[336:700]

In [9]:
orchid_2025 = alt.Chart(orchid_2025).mark_line(color='darkblue', strokeWidth=2).encode(
    alt.X('wtr_day_year'),
    alt.Y('depth'),
    tooltip=[alt.Tooltip('monthdate(date):T', title="Date"), alt.Tooltip('depth:Q', title="Snow Depth")]
)

In [10]:
orchid_analog = alt.Chart(orchid_2010).mark_line(color='darkgreen', strokeWidth=2).encode(
    alt.X('wtr_day_year'),
    alt.Y('depth'),
    tooltip=[alt.Tooltip('monthdate(date):T', title="Date"), alt.Tooltip('depth:Q', title="Snow Depth")]
)

In [11]:
alt.data_transformers.disable_max_rows()

title = alt.TitleParams(
   text='Orchid Lake Weather Station - 1175 m',
   subtitle="Snow Depth (2006/07 - 2023/24)",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

area = alt.Chart(orchid_stats, title=title).mark_area(color='#7DA6F3', opacity=0.5).encode(
    alt.X('wtr_day_year', title='Date - Oct 1(0) to Sep 30(365)', scale=alt.Scale(domain=[0,320], nice=False), axis=alt.Axis(values=[1,32,62,93,124,152,183,213,244,274,305,336])),
    alt.Y('max:Q', title='Snow Depth (cm)', scale=alt.Scale(domain=[0,700])),
    alt.Y2('min:Q')
).properties(width=600, height=300)

area2 = alt.Chart(orchid_stats).mark_area(color='#919397', opacity=0.6).encode(
    alt.X('wtr_day_year', scale=alt.Scale(domain=[0,320], nice=False), axis=alt.Axis(values=[1,32,62,93,124,152,183,213,244,274,305,336])),
    alt.Y('percentile_75th:Q'),
    alt.Y2('percentile_25th:Q')
)

wmean = alt.Chart(orchid).mark_line(color= "black", opacity=0.6, strokeDash=[4, 2]).encode(
    alt.X('wtr_day_year', title='Date - Oct 1(0) to Sep 30(365)', scale=alt.Scale(domain=[0,320], nice=False), axis=alt.Axis(values=[1,32,62,93,124,152,183,213,244,274,305,336])),
    alt.Y('mean(depth)', title='Snow Depth (cm)'),
    tooltip=[alt.Tooltip('monthdate(date):T', title="Date"), alt.Tooltip('mean:Q', title="Mean Depth") , alt.Tooltip('wtr_day_year', title="Day of Water Year")]
)

wyear_2024 = alt.Chart(orchid[orchid['wtr_year'] == 2024]).mark_line(color= "violet", opacity=0.6).encode(
    alt.X('wtr_day_year', title='Date - Oct 1(0) to Sep 30(365)', scale=alt.Scale(domain=['2022-10-01','2023-08-15'], nice=False), axis=alt.Axis(values=[1,32,62,93,124,152,183,213,244,274,305,336])),
    alt.Y('depth', title='Snow Depth (cm)')
)

complete = area + area2  + wmean + wyear_2024 + orchid_2025.interactive()


In [12]:
# Add text annotation
red_text = alt.Chart().mark_text(
    text='Red line = 2023/24',
    align='left',
    size=10,
    baseline='middle',
    color='darkred',
    dx=4,
    dy=10,
    font='Arial',  # Change the font to your preference
    fontWeight='normal',  # Adjust the weight (normal or bold)
    fontStyle='normal'  # Adjust the style (normal or italic)
).encode(
    x=alt.value(10),
    y=alt.value(10)
)

In [13]:
# Create Altair chart
test_2023 = alt.Chart(orchid).mark_line(color="violet", opacity=0.6).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-08-15'])),
    alt.Y('depth:Q', title='Snow Depth (cm)')
).properties(width=600)


In [14]:
per_text = alt.Chart().mark_text(
    text='62% of normal as of January 15',  # Replace with your desired text
    align='left',
    size=10,
    baseline='middle',
    color='darkblue',
    dx=2  # adjust the position of the text
).encode(
    x=alt.value(10),  # Adjust the x-position of the text box
    y=alt.value(10)   # Adjust the y-position of the text box
)

In [15]:
orchid_stats.head()

,date,depth,wtr_year,wtr_day_year,year,month,day,DOY,mean,max,min,std,sem,ci95_hi,ci95_lo,percentile_25th,percentile_75th
336,2008-10-01,0.0,2009,2022-10-01,2008,10,1,275,0.1,1.1,0.0,0.266789,0.064706,0.226824,-0.026824,0.0,0.0
337,2008-10-02,0.0,2009,2022-10-02,2008,10,2,276,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
338,2008-10-03,0.0,2009,2022-10-03,2008,10,3,277,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
339,2008-10-04,0.0,2009,2022-10-04,2008,10,4,278,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
340,2008-10-05,0.0,2009,2022-10-05,2008,10,5,279,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [16]:

alt.data_transformers.disable_max_rows()

title = alt.TitleParams(
   text='Orchid Lake Weather Station - 1175 m',
   subtitle="Dark gray is 25th to 75th percentile range. Light gray is range of max/min. Dashed line in mean.",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

area = alt.Chart(orchid_stats, title=title).mark_area(color='lightgray', opacity=0.5).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-08-15'])),
    alt.Y('max:Q', title='Snow Depth (cm)', scale=alt.Scale(domain=[0,700])),
    alt.Y2('min:Q')
).properties(width=600, height=300)

area2 = alt.Chart(orchid_stats).mark_area(color='#919397', opacity=0.6).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-08-15'])),
    alt.Y('percentile_75th:Q'),
    alt.Y2('percentile_25th:Q')
)

wmean = alt.Chart(orchid_stats).mark_line(color= "black", opacity=0.6, strokeDash=[4, 2]).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-08-15'])),
    alt.Y('mean:Q', title='Snow Depth (cm)'),
    tooltip=[alt.Tooltip('monthdate(date):T', title="Date"), alt.Tooltip('mean:Q', title="Mean Depth") , alt.Tooltip('wtr_day_year', title="Day of Water Year")]
)

wyear_2024 = alt.Chart(orchid[orchid['wtr_year'] == 2024]).mark_line(color= "blue", opacity=0.6, strokeWidth=1).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-08-15'])),
    alt.Y('depth', title='Snow Depth (cm)')
)

wyear_2023 = alt.Chart(orchid[orchid['wtr_year'] == 2023]).mark_line(color= "green", opacity=0.6, strokeWidth=1).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-08-15'])),
    alt.Y('depth', title='Snow Depth (cm)')
)

last_value = alt.Chart(last).mark_circle(color= "darkblue", size=40).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-08-15'])),
    alt.Y('depth', title='Snow Depth (cm)')
)


complete = area + area2 + wmean + orchid_2025 + last_value
complete

alt.LayerChart(...)

In [17]:
wmean2 = alt.Chart(orchid_stats).mark_line(color= "black", opacity=0.6, strokeDash=[4, 2]).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-08-15'])),
    alt.Y('mean:Q', title='Snow Depth (cm)'),
    tooltip=[alt.Tooltip('monthdate(date):T', title="Date"), alt.Tooltip('mean:Q', title="Mean Depth") , alt.Tooltip('wtr_day_year', title="Day of Water Year")]
)

wmean2

alt.Chart(...)

In [18]:
palisade = pd.read_csv('data/palisade_snow.csv', parse_dates=['date']) 
palisade['year'] = pd.DatetimeIndex(palisade['date']).year
palisade['month'] = pd.DatetimeIndex(palisade['date']).month
palisade['day'] = pd.DatetimeIndex(palisade['date']).day
palisade['DOY'] = pd.DatetimeIndex(palisade['date']).dayofyear
palisade['wtr_day_year'] = pd.to_datetime('2023-01-01') + pd.to_timedelta(palisade['wtr_day_year'] - 93, unit='D')
# Exclude current year for calculating stats
palisade_current = palisade.loc[palisade['wtr_year'] == 2025]
palisade = palisade.loc[palisade['wtr_year'] != 2025]

In [19]:
# Claculate statistics on mean temp column
palisade['mean'] = palisade.groupby('wtr_day_year')['depth'].transform('mean').round(1)
palisade['max'] = palisade.groupby('wtr_day_year')['depth'].transform('max')
palisade['min'] = palisade.groupby('wtr_day_year')['depth'].transform('min')
palisade['std'] = palisade.groupby('wtr_day_year')['depth'].transform('std')
palisade['sem'] = palisade.groupby('wtr_day_year')['depth'].transform('sem')
palisade['ci95_hi'] = palisade['mean'] + 1.96* palisade['sem']
palisade['ci95_lo'] = palisade['mean'] - 1.96* palisade['sem']
palisade['percentile_25th'] = palisade.groupby('wtr_day_year')['depth'].transform(lambda x: np.percentile(x, 25))
palisade['percentile_75th'] = palisade.groupby('wtr_day_year')['depth'].transform(lambda x: np.percentile(x, 75))
palisade.head()

,date,depth,wtr_year,wtr_day_year,year,month,day,DOY,mean,max,min,std,sem,ci95_hi,ci95_lo,percentile_25th,percentile_75th
0,2017-10-01,0.0,2018,2022-10-01,2017,10,1,274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-10-02,0.0,2018,2022-10-02,2017,10,2,275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-10-03,0.0,2018,2022-10-03,2017,10,3,276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-10-04,0.0,2018,2022-10-04,2017,10,4,277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-10-05,0.0,2018,2022-10-05,2017,10,5,278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# filter to remove non snow season
palisade = palisade[(palisade['wtr_day_year'] >= '2022-10-01') & (palisade['wtr_day_year'] <= '2023-07-01')]
palisade_current = palisade_current[(palisade_current['wtr_day_year'] >= '2022-10-01') & (palisade_current['wtr_day_year'] <= '2023-07-01')]

In [21]:
pal_last = palisade_current.tail(1)
pal_last

,date,depth,wtr_year,wtr_day_year,year,month,day,DOY
2625,2024-12-10,27.5,2025,2022-12-10,2024,12,10,345


In [22]:
palisade_stats = palisade.loc[336:700]
palisade_stats.head()

,date,depth,wtr_year,wtr_day_year,year,month,day,DOY,mean,max,min,std,sem,ci95_hi,ci95_lo,percentile_25th,percentile_75th
365,2018-10-01,0.0,2019,2022-10-01,2018,10,1,274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
366,2018-10-02,0.0,2019,2022-10-02,2018,10,2,275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
367,2018-10-03,0.0,2019,2022-10-03,2018,10,3,276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
368,2018-10-04,0.0,2019,2022-10-04,2018,10,4,277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
369,2018-10-05,0.0,2019,2022-10-05,2018,10,5,278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
palisade_2025 = alt.Chart(palisade_current).mark_line(color='darkblue', strokeWidth=2).encode(
    alt.X('wtr_day_year', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-07-01'])),
    alt.Y('depth:Q'),
    tooltip=[alt.Tooltip('monthdate(date):T', title="Date"), alt.Tooltip('depth:Q', title="Snow Depth")]
)
palisade_2025

alt.Chart(...)

In [24]:
alt.data_transformers.disable_max_rows()

title = alt.TitleParams(
   text='Palisade Lake Weather Station - 900 m',
   subtitle="2017-18 - 2024-25",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

area3 = alt.Chart(palisade_stats, title=title).mark_area(color='#7DA6F3', opacity=0.5).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-07-01'])),
    alt.Y('max:Q', title='Snow Depth (cm)', scale=alt.Scale(domain=[0,400])),
    alt.Y2('min:Q')
).properties(width=600, height=300)

area4 = alt.Chart(palisade_stats).mark_area(color='#919397', opacity=0.6).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-07-01'])),
    alt.Y('percentile_75th:Q'),
    alt.Y2('percentile_25th:Q')
)

pal_wmean = alt.Chart(palisade).mark_line(color= "black", opacity=0.6, strokeDash=[4, 2]).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-07-01'])),
    alt.Y('mean(depth)', title='Snow Depth (cm)'),
    tooltip=[alt.Tooltip('monthdate(date):T', title="Date"), alt.Tooltip('mean:Q', title="Mean Depth") , alt.Tooltip('wtr_day_year', title="Day of Water Year")]
)

palisade_2024 = alt.Chart(palisade[palisade['wtr_year'] == 2024]).mark_line(color= "blue", opacity=0.6, strokeWidth=1).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-07-01'])),
    alt.Y('depth', title='Snow Depth (cm)')
)

palisade_2023 = alt.Chart(palisade[palisade['wtr_year'] == 2023]).mark_line(color= "darkgreen", opacity=0.6, strokeWidth=1).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-07-01'])),
    alt.Y('depth', title='Snow Depth (cm)')
)

palisade_last = alt.Chart(pal_last).mark_circle(color= "darkblue", size=40).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-08-15'])),
    alt.Y('depth', title='Snow Depth (cm)')
)

complete_pal = area3 + area4  + pal_wmean + palisade_2024 + palisade_2023 + palisade_2025 + palisade_last
complete_pal


alt.LayerChart(...)

In [25]:
disap = pd.read_csv('data/disappointment_snow.csv', parse_dates=['date']) 
disap['year'] = pd.DatetimeIndex(disap['date']).year
disap['month'] = pd.DatetimeIndex(disap['date']).month
disap['day'] = pd.DatetimeIndex(disap['date']).day
disap['DOY'] = pd.DatetimeIndex(disap['date']).dayofyear

disap['wtr_day_year'] = pd.to_datetime('2023-01-01') + pd.to_timedelta(disap['wtr_day_year'] - 93, unit='D')
# Exclude current year for calculating stats
disap_current = disap.loc[disap['wtr_year'] == 2024]
disap = disap.loc[disap['wtr_year'] != 2024]

In [26]:
# Claculate statistics on mean temp column
disap['mean'] = disap.groupby('wtr_day_year')['depth'].transform('mean').round(1)
disap['max'] = disap.groupby('DOY')['depth'].transform('max')
disap['min'] = disap.groupby('DOY')['depth'].transform('min')
disap['std'] = disap.groupby('DOY')['depth'].transform('std')
disap['sem'] = disap.groupby('DOY')['depth'].transform('sem')
disap['ci95_hi'] = disap['mean'] + 1.96* disap['sem']
disap['ci95_lo'] = disap['mean'] - 1.96* disap['sem']
disap['percentile_25th'] = disap.groupby('wtr_day_year')['depth'].transform(lambda x: np.percentile(x, 25))
disap['percentile_75th'] = disap.groupby('wtr_day_year')['depth'].transform(lambda x: np.percentile(x, 75))

In [27]:
disap_stats = disap.loc[336:700]

In [28]:
disap_2024 = alt.Chart(disap_current).mark_line(color='darkblue', strokeWidth=2).encode(
    alt.X('wtr_day_year'),
    alt.Y('depth:Q'),
    tooltip=[alt.Tooltip('monthdate(date):T', title="Date"), alt.Tooltip('depth:Q', title="Snow Depth")]
)

In [29]:
alt.data_transformers.disable_max_rows()

title = alt.TitleParams(
   text='Disappointment Lake Weather Station - 1050 m',
   subtitle="Snow Depth (2015/16 - 2023/24)",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

area = alt.Chart(disap_stats, title=title).mark_area(color='#7DA6F3', opacity=0.5).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-08-01'])),
    alt.Y('max:Q', title='Snow Depth (cm)', scale=alt.Scale(domain=[0,500])),
    alt.Y2('min:Q')
).properties(width=600, height=300)

area2 = alt.Chart(disap_stats).mark_area(color='#919397', opacity=0.6).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-08-01'])),
    alt.Y('percentile_75th:Q'),
    alt.Y2('percentile_25th:Q')
)

disap_wmean = alt.Chart(disap).mark_line(color= "black", opacity=0.6, strokeDash=[4, 2]).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-08-01'])),
    alt.Y('mean', title='Snow Depth (cm)'),
    tooltip=[alt.Tooltip('monthdate(date):T', title="Date"), alt.Tooltip('mean:Q', title="Mean Depth") , alt.Tooltip('wtr_day_year', title="Day of Water Year")]
)

disap_wyear_2023 = alt.Chart(disap[disap['wtr_year'] == 2023]).mark_line(color= "violet", opacity=0.6).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d', tickCount=12, labelOverlap='parity'), scale=alt.Scale(domain=['2022-10-01','2023-08-01'])),
    alt.Y('depth', title='Snow Depth (cm)')
)

area + area2  + disap_wmean + disap_wyear_2023 + disap_2024

alt.LayerChart(...)

### Satellite-derived Snow Water Equivalent
The images below show the average SWE over the Capilano and Seymour Watersheds as of November 29th. Values are in mm of snow water equivalent. 

```{figure} img/sheds_swe.png
---
name: sheds_swe
---
Average SWE in Capilano and Seymour Watersheds - Nov. 29, 2023
```

## Snowpack Data and Climate Indices
The plots below look at snow water equivalent (SWE) at Grouse Mountain on April 1, which is typically close to peak SWE for the season, in relation to the El Nino - Southern Oscillation (ENSO) and Pacific Decadal Oscillation (PDO) climate indices. Both of these climate drivers have a significant impact on spring SWE values in the watersheds. However, there is also significant variability. Fortunately, the PDO remains strongly negative as we move into 2024. Perhaps this will dampen the impacts of a moderate-to-strong El Nino season. 

In [30]:
snow_df = pd.read_csv('data/snow_data_full.csv')
snow_df['date'] = pd.to_datetime(snow_df[['year', 'month', 'day']])
snow_df['year'] = pd.DatetimeIndex(snow_df['date']).year
snow_df['month'] = pd.DatetimeIndex(snow_df['date']).month
snow_df['DOY'] = pd.DatetimeIndex(snow_df['date']).dayofyear


In [31]:
snow_df['mean_depth'] = snow_df[['grouse_depth', 'dog_depth', 'orchid_depth', 'disappointment_depth', 'palisade_depth']].mean(axis=1).round(1)
snow_df['mean_swe'] = snow_df[['grouse_swe', 'dog_swe', 'orchid_swe', 'disappointment_swe', 'palisade_swe']].mean(axis=1).round(1)
snow_df['mean_density'] = snow_df[['grouse_density', 'dog_density', 'orchid_density', 'disappointment_density', 'palisade_density']].mean(axis=1).round(1)

In [32]:
# Calculate SWE for a month for a site
apr_swe = snow_df.loc[snow_df['month'] == 4]
#apr_swe = apr_swe.loc[apr_swe['day'] == 15]
apr_swe.head(5) 
# Pick a site
grouse_apr_swe = apr_swe.loc[:, ['grouse_depth', 'grouse_swe', 'grouse_density', 'date', 'pdo_phase_main', 'enso_oni_main']]
#grouse_apr_swe = grouse_apr_swe[(grouse_apr_swe['date'] > '1950-01-01')]

In [33]:
grouse_apr_swe['grouse_moving_avg'] = grouse_apr_swe['grouse_swe'].rolling(10).mean()
#palisade_may_swe['palisade_moving_avg'] = palisade_may_swe['palisade_swe'].rolling(10).mean()
#grouse_may_swe['grouse_moving_avg'] = grouse_may_swe['grouse_swe'].rolling(10).mean()
#dog_may_swe['dog_moving_avg'] = dog_may_swe['dog_swe'].rolling(10).mean()


In [34]:
domain = ['negative', 'positive']
range_ = ['blue', 'red']

title4 = alt.TitleParams(
   text='Grouse Mountain Snow Course (3A01)',
   subtitle="April 1 SWE by PDO Phase",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

line_grouse_apr = alt.Chart(grouse_apr_swe).mark_line(strokeDash=[4, 2], color='black', opacity=0.7).encode(
    alt.X('date:T'),
    alt.Y('grouse_moving_avg:Q')
)

bar_chart2 = alt.Chart(grouse_apr_swe, title=title4).mark_bar(opacity=0.8).encode(
    alt.X('date:T', title='', axis=alt.Axis(grid=False)),
    alt.Y('grouse_swe:Q', title='Snow Water Equivalent (mm)'),
    alt.Color('pdo_phase_main', title='PDO Phase', scale=alt.Scale(domain=domain, range=range_)),
    tooltip=[alt.Tooltip('date:T', title="Date"), alt.Tooltip('grouse_swe:Q', title="Snow Depth")]
).properties(width=600)

chart2 = bar_chart2 + line_grouse_apr
chart2


alt.LayerChart(...)

In [35]:
pdo_avg = grouse_apr_swe.groupby("pdo_phase_main")[["grouse_swe"]].describe()
pdo_avg = pdo_avg.reset_index()


In [36]:
domain = ['negative', 'neutral', 'positive']
range_ = ['blue', 'gray', 'red']

title5 = alt.TitleParams(
   text='Grouse Mountain Snow Course (3A01)',
   subtitle="April 1 SWE by ENSO Phase",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

climate_chart2 = alt.Chart(grouse_apr_swe, title=title5).mark_bar(opacity=0.8).encode(
    alt.X('date:T', title=''),
    alt.Y('grouse_swe:Q', title='Snow Water Equivalent (mm)'),
    alt.Color('enso_oni_main', title='ENSO Phase',scale=alt.Scale(domain=domain, range=range_)),
    tooltip=[alt.Tooltip('date:T', title="Date"), alt.Tooltip('grouse_swe:Q', title="Snow Depth")]
).properties(width=600)

climate_chart2 = climate_chart2 + line_grouse_apr
climate_chart2

alt.LayerChart(...)

In [37]:
enso_avg = grouse_apr_swe.groupby("enso_oni_main")[["grouse_swe"]].describe()
enso_avg = enso_avg.reset_index()
# Flatten the multi-level column names
columns_flattened = ['{}_{}'.format(col[0], col[1]) if col[1] else col[0] for col in enso_avg.columns]
enso_avg.columns = columns_flattened

# Replace special characters in column names
columns_replaced = [col.replace('%', 'percent').replace('.', '_') for col in enso_avg.columns]
enso_avg.columns = columns_replaced

In [38]:
domain = ['negative', 'neutral', 'positive']
range_ = ['blue', 'gray', 'red']
# Boxplot
enso_boxplot = alt.Chart(grouse_apr_swe).mark_boxplot(extent="min-max", opacity=0.8, size=20).encode(
    alt.X('enso_oni_main:O', title=None),
    alt.Y('grouse_swe:Q', title=None),
    tooltip=alt.Tooltip(['mean(grouse_swe)'], format=',.1f'),
    color=alt.Color("enso_oni_main:N", title='ENSO Phase', legend=None, scale=alt.Scale(domain=domain, range=range_))
).properties(
    width=250, height=200, title='SWE Distribution by ENSO Phase'  # You can adjust the width of the plot as needed
)

In [39]:
domain = ['negative', 'positive']
range_ = ['blue', 'red']
# Boxplot
pdo_boxplot = alt.Chart(grouse_apr_swe).mark_boxplot(extent="min-max",opacity=0.8, size=20).encode(
    alt.X('pdo_phase_main:O', title=None),
    alt.Y('grouse_swe:Q', title='SWE (mm)'),
    tooltip=alt.Tooltip(['mean(grouse_swe)'], format=',.1f'),
    color=alt.Color("pdo_phase_main:N", title='PDO Phase', legend=None, scale=alt.Scale(domain=domain, range=range_))
).properties(
    width=250, height=200, title='SWE Distribution by PDO Phase'  # You can adjust the width of the plot as needed
)

In [40]:
combined_box = pdo_boxplot | enso_boxplot.resolve_scale(color='independent')
combined_box.configure_axisX(labelAngle=0)

alt.HConcatChart(...)

In [41]:
wegaw_sey = pd.read_csv('data/wegaw_seymour_swe.csv', parse_dates=['date']) 

wegaw_sey['year'] = pd.DatetimeIndex(wegaw_sey['date']).year
wegaw_sey['month'] = pd.DatetimeIndex(wegaw_sey['date']).month
wegaw_sey['day'] = pd.DatetimeIndex(wegaw_sey['date']).day
wegaw_sey['DOY'] = pd.DatetimeIndex(wegaw_sey['date']).dayofyear
# Assuming 'wtr_day_year' is currently in integer format
wegaw_sey['wtr_day_year'] = pd.to_datetime('2023-01-01') + pd.to_timedelta(wegaw_sey['wtr_day_year'] - 93, unit='D')

wegaw_sey.head()

,date,SWE,swe_mm,wtr_day_year,wtr_year,year,month,day,DOY
0,2018-10-01,0.000000,0.000000,2022-10-01,2019,2018,10,1,274
1,2018-10-02,0.000000,0.000000,2022-10-02,2019,2018,10,2,275
2,2018-10-03,0.000000,0.000000,2022-10-03,2019,2018,10,3,276
3,2018-10-04,0.852511,8.525110,2022-10-04,2019,2018,10,4,277
4,2018-10-05,0.803844,8.038443,2022-10-05,2019,2018,10,5,278


In [42]:
swe_line = alt.Chart(wegaw_sey[wegaw_sey['wtr_year'] != 2024]).mark_line(opacity=0.7, strokeWidth=1).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d'), scale=alt.Scale(domain=['2022-10-01','2023-09-30'])),
    alt.Y('swe_mm:Q', title='Snow Water Equivalent (mm)'),
    alt.Color('wtr_year:O', title='Water Year', legend=alt.Legend(orient="bottom"), scale=alt.Scale(scheme='viridis'))
)
mean_line = alt.Chart(wegaw_sey).mark_line(opacity=0.7, strokeDash=[3,3], color='black').encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d'), scale=alt.Scale(domain=['2022-10-01','2023-09-30'])),
    alt.Y('mean(swe_mm):Q', title='Snow Water Equivalent (mm)')
)
year_2024 = alt.Chart(wegaw_sey[wegaw_sey['wtr_year'] == 2024]).mark_line(color='darkblue', strokeWidth=2).encode(
    alt.X('wtr_day_year:T', title=None,
          axis=alt.Axis(format='%b-%d'), scale=alt.Scale(domain=['2022-10-01','2023-09-30'])),
    alt.Y('swe_mm:Q', title='Snow Water Equivalent (mm)')
)

complete_chart = year_2024 + swe_line + mean_line
complete_chart.properties(title='Basin Average Snow Water Equivalent - Seymour', width=700, height=300)

alt.LayerChart(...)